# US Cities and Zonal Stats Workflow Example

## Intro

This notebook provides a quick walk through on how the Sust Global physical risk dataset can be used to make a heatmap based on a collection of asset locations. 

We use a sample dataset of [populous US Cities](https://github.com/plotly/datasets/blob/master/us-cities-top-1k.csv) and the zonal climate risk stats from Sust Global to show case an example heatmap created with the folium python library.

In [1]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium import Map
import json
import contextily as ctx
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rio 
from shapely.geometry import Polygon, MultiPolygon, shape, Point

## Helper Functions

In [2]:
def folium_heatmap(df, hazard):
    index=0
    m = folium.Map([df['lat'].values[0], df['lng'].values[0]], zoom_start=4, tiles='cartodbpositron')
    for point in range(0, df.shape[0]):
            tooltip_label = str(df['Entity Name'].values[point]) 
            if pd.isnull(df['lat'].values[point]) or pd.isnull(df['lng'].values[point]):
                print('Null/NaN values')
            else:
                scenario_analytics = json.loads(df['scenario_analytics'].values[point])
                label = scenario_analytics['ssp585'][hazard]['summary_label']
                if label == 'HIGH':
                    color = 'red'
                elif label == 'MEDIUM':
                    color = 'yellow'
                else:
                    color = 'green'
                folium.CircleMarker([df['lat'].values[point],df['lng'].values[point]], radius=3, color=color, fill=True, fill_opacity=0.8,fill_color=color, popup=tooltip_label).add_to(m)

    return m

## Loading data

Loading data from the US cities sample dataset.

In [3]:
df = pd.read_csv('./data/USCities.csv', low_memory=False)
pd.set_option('display.max_rows', 1000)
print(df.columns)
print(df.shape)
display(df.head(10))

Index(['City', 'State', 'Population', 'lat', 'lng'], dtype='object')
(1000, 5)


,City,State,Population,lat,lng
0,Marysville,Washington,63269,48.051764,-122.177082
1,Perris,California,72326,33.782519,-117.228648
2,Cleveland,Ohio,390113,41.499320,-81.694361
3,Worcester,Massachusetts,182544,42.262593,-71.802293
4,Columbia,South Carolina,133358,34.000710,-81.034814
5,Waterbury,Connecticut,109676,41.558153,-73.051496
6,Eagan,Minnesota,65453,44.804132,-93.166886
7,Southfield,Michigan,73006,42.473369,-83.221873
8,Lafayette,Louisiana,124276,30.224090,-92.019843
9,Boise City,Idaho,214237,43.618710,-116.214607


# Simple Queries on US Cities data

- How many cities exist across the dataset?
- What is the most populous city?

In [4]:
len(df)

1000

In [5]:
df.nlargest(10, 'Population')

,City,State,Population,lat,lng
890,New York,New York,8405837,40.712784,-74.005941
953,Los Angeles,California,3884307,34.052234,-118.243685
594,Chicago,Illinois,2718782,41.878114,-87.629798
646,Houston,Texas,2195914,29.760427,-95.369803
53,Philadelphia,Pennsylvania,1553165,39.952584,-75.165222
966,Phoenix,Arizona,1513367,33.448377,-112.074037
153,San Antonio,Texas,1409019,29.424122,-98.493628
333,San Diego,California,1355896,32.715738,-117.161084
59,Dallas,Texas,1257676,32.776664,-96.796988
805,San Jose,California,998537,37.338208,-121.886329


# Transform to a geodataframe

In [6]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat))
gdf = gdf.set_crs("epsg:4326", inplace = True)
print(gdf.crs)
print(gdf.shape)

epsg:4326
(1000, 6)


# Load up zonal stats dataset from Sust Global

Use admin 2 for the US here (counties).

In [7]:
src = './data/USA_2.geojson'

gdf_counties = gpd.read_file(src)
print(gdf_counties.shape)
print(gdf_counties.crs)
display(gdf_counties[:10])

(3234, 14)
EPSG:4326


,scenario_analytics,id,entity_id,entity_name,labels,lng,lat,admin_processing_level,admin0,admin1,admin2,admin3,admin4,geometry
0,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",0,,,"{ ""label:fid"": 1, ""label:LSAD"": ""06"", ""label:N...",-81.74317,32.39681,2,USA,Georgia,13031,,,"POLYGON ((-82.02684 32.55516, -82.03023 32.538..."
1,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",1,,,"{ ""label:fid"": 2, ""label:LSAD"": ""06"", ""label:N...",-84.46701,33.79024,2,USA,Georgia,13121,,,"POLYGON ((-84.84931 33.51318, -84.85071 33.511..."
2,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",10,,,"{ ""label:fid"": 11, ""label:LSAD"": ""06"", ""label:...",-83.56637,34.13387,2,USA,Georgia,13157,,,"POLYGON ((-83.81768 34.12749, -83.816 34.12688..."
3,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",100,,,"{ ""label:fid"": 101, ""label:LSAD"": ""06"", ""label...",-83.42110,38.19625,2,USA,Kentucky,21205,,,"POLYGON ((-83.63517 38.19025, -83.6352 38.1875..."
4,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",1000,,,"{ ""label:fid"": 1001, ""label:LSAD"": ""06"", ""labe...",-123.09834,45.56007,2,USA,Oregon,41067,,,"POLYGON ((-123.48544 45.44713, -123.48608 45.4..."
5,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",1001,,,"{ ""label:fid"": 1002, ""label:LSAD"": ""06"", ""labe...",-118.88687,37.93910,2,USA,California,06051,,,"POLYGON ((-119.64894 38.28913, -119.65137 38.2..."
6,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",1002,,,"{ ""label:fid"": 1003, ""label:LSAD"": ""06"", ""labe...",-91.10702,36.04122,2,USA,Arkansas,05075,,,"POLYGON ((-91.35694 35.90519, -91.35742 35.890..."
7,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",1003,,,"{ ""label:fid"": 1004, ""label:LSAD"": ""06"", ""labe...",-93.66842,33.73538,2,USA,Arkansas,05057,,,"POLYGON ((-93.96945 33.74021, -93.96856 33.737..."
8,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",1004,,,"{ ""label:fid"": 1005, ""label:LSAD"": ""06"", ""labe...",-94.27433,35.19884,2,USA,Arkansas,05131,,,"POLYGON ((-94.4476 34.94192, -94.44789 34.9340..."
9,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",1005,,,"{ ""label:fid"": 1006, ""label:LSAD"": ""06"", ""labe...",-120.51611,39.58037,2,USA,California,06091,,,"POLYGON ((-121.05748 39.53999, -121.0582 39.53..."


In [8]:
gdf_counties['scenario_analytics'][0]

'{ "ssp585": { "wildfire": { "summary_score": 0, "summary_label": "LOW", "indicator_baseline": 0, "indicator_baseline_lbd": 0, "indicator_baseline_ubd": 0, "indicator_2030": 0, "indicator_2030_lbd": 0, "indicator_2030_ubd": 0, "indicator_2050": 0, "indicator_2050_lbd": 0, "indicator_2050_ubd": 0, "indicator_2080": 0, "indicator_2080_lbd": 0, "indicator_2080_ubd": 0, "structural_damage_baseline": 0, "structural_damage_baseline_lbd": 0, "structural_damage_baseline_ubd": 0, "structural_damage_2030": 0, "structural_damage_2030_lbd": 0, "structural_damage_2030_ubd": 0, "structural_damage_2050": 0, "structural_damage_2050_lbd": 0, "structural_damage_2050_ubd": 0, "structural_damage_2080": 0, "structural_damage_2080_lbd": 0, "structural_damage_2080_ubd": 0, "business_interruption_baseline": 0, "business_interruption_baseline_lbd": 0, "business_interruption_baseline_ubd": 0, "business_interruption_2030": 0, "business_interruption_2030_lbd": 0, "business_interruption_2030_ubd": 0, "business_int

# Run a spatial join to get risk metadata for each property 

The spatial join will append risk data to the asset dataframe depending on the zone they fall in.

In [9]:
gdf_risk = gpd.sjoin(gdf, gdf_counties, predicate='within')
print(gdf_risk.shape)
gdf_risk.rename(columns={'lat_left':'lat', 'lng_left':'lng', 'City':'Entity Name'}, inplace=True)
display(gdf_risk.head(10))

(1000, 20)


,Entity Name,State,Population,lat,lng,geometry,index_right,scenario_analytics,id,entity_id,entity_name,labels,lng_right,lat_right,admin_processing_level,admin0,admin1,admin2,admin3,admin4
0,Marysville,Washington,63269,48.051764,-122.177082,POINT (-122.17708 48.05176),3071,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",852,,,"{ ""label:fid"": 853, ""label:LSAD"": ""06"", ""label...",-121.69719,48.04736,2,USA,Washington,53061,,
1,Perris,California,72326,33.782519,-117.228648,POINT (-117.22865 33.78252),147,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",113,,,"{ ""label:fid"": 114, ""label:LSAD"": ""06"", ""label...",-115.99385,33.74368,2,USA,California,06065,,
2,Cleveland,Ohio,390113,41.499320,-81.694361,POINT (-81.69436 41.49932),3011,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",799,,,"{ ""label:fid"": 800, ""label:LSAD"": ""06"", ""label...",-81.65860,41.42430,2,USA,Ohio,39035,,
3,Worcester,Massachusetts,182544,42.262593,-71.802293,POINT (-71.80229 42.26259),1724,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",255,,,"{ ""label:fid"": 256, ""label:LSAD"": ""06"", ""label...",-71.90774,42.35142,2,USA,Massachusetts,25027,,
4,Columbia,South Carolina,133358,34.000710,-81.034814,POINT (-81.03481 34.00071),2914,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",710,,,"{ ""label:fid"": 711, ""label:LSAD"": ""06"", ""label...",-80.90310,34.02185,2,USA,South Carolina,45079,,
5,Waterbury,Connecticut,109676,41.558153,-73.051496,POINT (-73.0515 41.55815),324,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 8...",129,,,"{ ""label:fid"": 130, ""label:LSAD"": ""06"", ""label...",-72.93219,41.41063,2,USA,Connecticut,09009,,
6,Eagan,Minnesota,65453,44.804132,-93.166886,POINT (-93.16689 44.80413),316,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",1282,,,"{ ""label:fid"": 1283, ""label:LSAD"": ""06"", ""labe...",-93.06544,44.67188,2,USA,Minnesota,27037,,
7,Southfield,Michigan,73006,42.473369,-83.221873,POINT (-83.22187 42.47337),3131,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",906,,,"{ ""label:fid"": 907, ""label:LSAD"": ""06"", ""label...",-83.38580,42.66039,2,USA,Michigan,26125,,
8,Lafayette,Louisiana,124276,30.224090,-92.019843,POINT (-92.01984 30.22409),2921,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",717,,,"{ ""label:fid"": 718, ""label:LSAD"": ""15"", ""label...",-92.06391,30.20683,2,USA,Louisiana,22055,,
9,Boise City,Idaho,214237,43.618710,-116.214607,POINT (-116.21461 43.61871),2143,"{ ""ssp585"": { ""wildfire"": { ""summary_score"": 0...",2927,,,"{ ""label:fid"": 2928, ""label:LSAD"": ""06"", ""labe...",-116.24116,43.45112,2,USA,Idaho,16001,,


# Plot on slippy map as heatmap

You can pass hazard value as one of the following: 'wildfire', 'flood', 'cyclone', 'heatwave', 'sea_level_rise', 'water_stress'.

In [10]:
m = folium_heatmap(gdf_risk, 'cyclone')
m